# 7.5 IMDb(Internet Movie Database)에서 DataLoader 작성

- IMDb(Internet Movie Database) 데이터를 사용하여, 감정 분석(0: negative, 1: positive)을 두 값 클래스 분류하기 위한 Dataset과 DataLoader를 작성합니다.


※ 이 장의 파일은 Ubuntu 환경에서의 동작을 전제로 하고 있습니다. Windows와 같이 문자 코드가 다른 환경에서는 동작에 주의하십시오.

# 7.5 학습 목표

1.	텍스트 파일 데이터로 tsv 파일을 작성하고, torchtext용 DataLoader를 만들 수 있다

# 사전 준비
도서의 지시에 따라, 이 장에서 사용하는 데이터를 준비합니다


# 1. IMDb 데이터 세트를 tsv 형식으로 변환

Dataset을 다운로드합니다

※torchtext로 IMDb를 사용할 수 있는 함수가 있지만, 데이터 세트가 준비되어 있지 않은 경우에도 대응할 수 있도록 처음부터 작성합니다.

http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

5만건의 데이터(train, test는 2.5만건씩)입니다. 데이터 id와 rating(1~10)으로 파일명이 정해져 있습니다.

rate는 10이 최고이고, 4이하는 negative, 7 이상은 positive로 클래스가 나누어져 있습니다.



In [1]:
# tsv 형식의 파일입니다
import glob
import os
import io
import string


# 훈련 데이터의 tsv 파일을 작성합니다
f = open('./data/IMDb_train.tsv', 'w')

path = './data/aclImdb/train/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # 탭은 지웁니다
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)

path = './data/aclImdb/train/neg/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # 탭은 지웁니다
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()


In [2]:
# 테스트 데이터 작성
f = open('./data/IMDb_test.tsv', 'w')

path = './data/aclImdb/test/pos/'
for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # 탭은 지웁니다
        text = text.replace('\t', " ")

        text = text+'\t'+'1'+'\t'+'\n'
        f.write(text)


path = './data/aclImdb/test/neg/'

for fname in glob.glob(os.path.join(path, '*.txt')):
    with io.open(fname, 'r', encoding="utf-8") as ff:
        text = ff.readline()

        # 탭은 지웁니다
        text = text.replace('\t', " ")

        text = text+'\t'+'0'+'\t'+'\n'
        f.write(text)

f.close()


# 2. 전처리 및 단어 분할 함수 정의

In [3]:
import string
import re

# 다음 기호는 스페이스(공백)으로 치환합니다(쉼표, 마침표 제외).
# punctuation은 구두점입니다
print("구두점 문자: ", string.punctuation)
# !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~

# 전처리
def preprocessing_text(text):
    # 개행 코드 삭제
    text = re.sub('<br />', '', text)

    # 쉼표, 마침표 이외의 기호를 공백으로 치환
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    # 쉼표, 마침표의 전후에 공백 추가
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text

# 띄어쓰기(이번에는 영어 데이터이며, 임시로 공백으로 구분)
def tokenizer_punctuation(text):
    return text.strip().split()


# 전처리 및 띄어쓰기를 포함한 함수 정의
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)
    ret = tokenizer_punctuation(text)
    return ret


# 동작을 확인합니다
print(tokenizer_with_preprocessing('I like cats.'))


구두점 문자:  !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['I', 'like', 'cats', '.']


# DataLoader 작성

In [4]:
# 데이터를 읽었을 때, 내용에 대해 수행할 처리를 정의합니다
import torchtext


# 문장과 라벨을 모두 준비합니다
max_length = 256
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="<cls>", eos_token="<eos>")
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# 인수의 의미는 다음과 같습니다
# init_token: 전체 문장의 처음에 넣는 단어
# eos_token: 전체 문장의 끝에 넣는 단어


In [5]:
# "data" 폴더에서 각 tsv 파일을 읽어들입니다
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# 동작 확인
print('훈련 및 검증 데이터 수', len(train_val_ds))
print('첫번째 훈련 및 검증 데이터', vars(train_val_ds[0]))


훈련 및 검증 데이터 수 25000
첫번째 훈련 및 검증 데이터 {'Text': ['i', 'couldn', 't', 'believe', 'the', 'comments', 'made', 'about', 'the', 'movie', '.', 'as', 'i', 'read', 'the', 'awful', 'opinions', 'about', 'the', 'movie', 'i', 'actually', 'wondered', 'if', 'you', 'had', 'actually', 'viewed', 'the', 'same', 'movie', 'that', 'i', 'did', '.', 'what', 'i', 'viewed', 'was', 'incredible', 'i', 'think', 'the', 'actresses', 'and', 'director', 'did', 'a', 'fantastic', 'job', 'in', 'the', 'movie', '.', 'i', 'hadn', 't', 'had', 'the', 'pleasure', 'to', 'see', 'either', 'actress', 'previously', 'and', 'i', 'couldn', 't', 'have', 'been', 'more', 'set', 'back', 'by', 'the', 'incredible', 'job', 'that', 'they', 'did', 'i', 'd', 'have', 'to', 'say', 'its', 'the', 'most', 'believable', 'movie', 'that', 'i', 've', 'seen', 'in', 'a', 'long', 'time', '.', 'what', 'i', 'don', 't', 'see', 'is', 'why', 'everyone', 'has', 'such', 'a', 'problem', 'with', 'deanna', 's', 'choice', 'of', 'drug', 'in', 'the', 'attempt', 'of', 'su

In [6]:
import random
# torchtext.data.Dataset의 split 함수로 훈련 데이터와 validation 데이터를 나눔

train_ds, val_ds = train_val_ds.split(
    split_ratio=0.8, random_state=random.seed(1234))

# 동작 확인
print('훈련 데이터의 수', len(train_ds))
print('검증 데이터의 수', len(val_ds))
print('첫번째 훈련 데이터', vars(train_ds[0]))


훈련 데이터의 수 20000
검증 데이터의 수 5000
첫번째 훈련 데이터 {'Text': ['i', 'watched', 'the', 'entire', 'movie', 'recognizing', 'the', 'participation', 'of', 'william', 'hurt', ',', 'natascha', 'mcelhone', ',', 'and', 'desiree', 'nosbusch', '.', 'i', 'm', 'glad', 'that', 'i', 'had', 'no', 'idea', 'of', 'the', 'presence', 'of', 'peter', 'weller', '.', 'at', 'the', 'end', 'of', 'the', 'movie', 'i', 'said', 'that', 'was', 'peter', 'weller', 'kudos', 'to', 'mr', '.', 'weller', 'for', 'an', 'outstanding', 'performance', '.', 'weller', 'played', 'a', 'major', 'character', ',', 'and', 'his', 'performance', 'was', 'such', 'that', 'i', 'didn', 't', 'even', 'recognize', 'him', '.', 'overall', 'the', 'plot', 'was', 'bad', ',', 'the', 'writing', 'was', 'bad', ',', 'and', 'the', 'performances', ',', 'aside', 'from', 'those', 'of', 'nosbusch', 'and', 'weller', ',', 'were', 'subpar', '.', 'the', 'scenery', 'and', 'setting', 'were', 'interesting', ',', 'and', 'weller', 'was', 'amazing', '.', '4', 'stars', ',', 'of', 'a'

# vocabulary 작성

In [7]:
# torchtext로 단어 벡터로서 학습된 모델(영어)을 읽어들입니다
from torchtext.vocab import Vectors

english_fasttext_vectors = Vectors(name='data/wiki-news-300d-1M.vec')


# 단어 벡터의 내용을 확인합니다
print("한 단어를 표현하는 차원 수: ", english_fasttext_vectors.dim)
print("단어 수: ", len(english_fasttext_vectors.itos))


100%|█████████▉| 999913/999994 [02:16<00:00, 8051.71it/s]

한 단어를 표현하는 차원 수:  300
단어 수:  999994


100%|█████████▉| 999913/999994 [02:31<00:00, 8051.71it/s]

In [8]:
# 벡터화 된 버전의 vocabulary를 만듭니다
TEXT.build_vocab(train_ds, vectors=english_fasttext_vectors, min_freq=10)

# vocabulary 벡터를 확인합니다
print(TEXT.vocab.vectors.shape)  # 17916개의 단어가 300차원 벡터로 표현되어 있음
TEXT.vocab.vectors

# vocabulary 단어 순서를 확인
TEXT.vocab.stoi


torch.Size([17916, 300])


defaultdict(<function torchtext.vocab._default_unk_index()>,
            {'<unk>': 0,
             '<pad>': 1,
             '<cls>': 2,
             '<eos>': 3,
             'the': 4,
             '.': 5,
             ',': 6,
             'and': 7,
             'a': 8,
             'of': 9,
             'to': 10,
             'is': 11,
             'it': 12,
             'in': 13,
             'i': 14,
             'this': 15,
             'that': 16,
             's': 17,
             'was': 18,
             'as': 19,
             'for': 20,
             'with': 21,
             'movie': 22,
             'but': 23,
             'film': 24,
             't': 25,
             'you': 26,
             'on': 27,
             'not': 28,
             'he': 29,
             'are': 30,
             'his': 31,
             'have': 32,
             'be': 33,
             'one': 34,
             'all': 35,
             'at': 36,
             'they': 37,
             'by': 38,
             'an': 3

In [9]:
# DataLoader를 작성합니다(torchtext에서는 단순히 iterater로 불립니다)
train_dl = torchtext.data.Iterator(train_ds, batch_size=24, train=True)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=24, train=False, sort=False)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=24, train=False, sort=False)


# 동작 확인(검증 데이터의 데이터 세트로 확인)
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)


(tensor([[  2,  15,  22,  ...,   1,   1,   1],
        [  2,  57,  14,  ...,   1,   1,   1],
        [  2,  14,  43,  ...,   1,   1,   1],
        ...,
        [  2,  15, 621,  ...,   1,   1,   1],
        [  2,  93,  25,  ...,   1,   1,   1],
        [  2,  15,  22,  ...,   1,   1,   1]]), tensor([122, 152, 167, 145, 104, 128, 238,  88, 256, 172, 205, 243, 256, 129,
        256, 198, 191, 172, 256, 256, 119, 173,  95, 141]))
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0])


이렇게 DataLoader는 단어 id를 저장하고 있으므로, 분산 표현은 딥러닝 모델 측에서 id에 따라 취득할 필요가 있습니다.

지금까지의 내용을 "utils" 폴더의 dataloader.py에 별도로 저장하고, 다음 절에서 import하여 사용합니다.

끝